In [ ]:
nn = 100
kmeans_model_path = "train_kmeans_{0}_clusters_model.m".format(class_num)

from sklearn.cluster import KMeans
from sklearn.externals import joblib

def trainKmeansClf(class_num, train_data, kmeans_model_path):
    kmeans_clf = KMeans(n_clusters=class_num, random_state=0).fit(train_data)
    joblib.dump(kmeans, kmeans_model_path)
    return kmeans_clf

def getKmeansClfModel(kmeans_model_path):
    return joblib.load(kmeans_model_path)

def 

In [ ]:
def _main():
    global trainKmeansClf
    global kmeans_model_path
    kmeans_clf = trainKmeansClf(CLASS_NUM, kmeans_model_path)

if "__main__" == __name__:
    _main()